In [4]:
import os
import json
import pickle
import networkx as nx
from tqdm import tqdm
from pycocotools.coco import COCO
import cv2
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import math
from py2neo import Graph, Node, Relationship

In [ ]:
# === CẤU HÌNH ĐƯỜNG DẪN ===

In [2]:
# Đường dẫn đến file annotation
annotation_file = 'E:/Download/annotations/instances_val2017.json'  

# Load COCO annotation
coco = COCO(annotation_file)

loading annotations into memory...
Done (t=0.84s)
creating index...
index created!


In [ ]:
# === LƯU LẠI NODE VÀO FILE CSV ===

In [5]:
def save_nodes_to_csv(coco, output_file="coco_nodes.csv"):
    all_nodes = []

    for image_id in coco.getImgIds():
        ann_ids = coco.getAnnIds(imgIds=image_id)
        anns = coco.loadAnns(ann_ids)

        for ann in anns:
            x1, y1, w, h = ann["bbox"]
            x2, y2 = x1 + w, y1 + h
            label = coco.loadCats(ann["category_id"])[0]["name"]
            node_id = ann["id"]

            all_nodes.append({
                "image_id": image_id,
                "node_id": node_id,
                "label": label,
                "x1": x1, "y1": y1, "x2": x2, "y2": y2
            })

    df_nodes = pd.DataFrame(all_nodes)
    df_nodes.to_csv(output_file, index=False)
    print(f"Đã lưu {len(df_nodes)} nodes vào: {output_file}")

save_nodes_to_csv(coco)

Đã lưu 36781 nodes vào: coco_nodes.csv


In [ ]:
# === HÀM XÁC ĐỊNH QUAN HỆ GIỮA CÁC NODE ===

In [6]:
# Load dữ liệu từ file nodes
df_nodes = pd.read_csv("coco_nodes.csv")

# Hàm tính khoảng cách giữa hai đối tượng
def calculate_distance(center_A, center_B):
    return math.sqrt((center_A[0] - center_B[0])**2 + (center_A[1] - center_B[1])**2)

# Hàm xác định quan hệ spatial giữa 2 objects
def determine_relationship(obj1, obj2):
    center_A = ((obj1["x1"] + obj1["x2"]) / 2, (obj1["y1"] + obj1["y2"]) / 2)
    center_B = ((obj2["x1"] + obj2["x2"]) / 2, (obj2["y1"] + obj2["y2"]) / 2)

    distance = calculate_distance(center_A, center_B)

    if obj1["y1"] < obj2["y1"] and obj1["y2"] > obj2["y2"] and obj1["x1"] < obj2["x1"] and obj1["x2"] > obj2["x2"]:
        return "ON"
    elif abs(center_A[0] - center_B[0]) < 50 and abs(center_A[1] - center_B[1]) < 50:
        return "NEXT_TO"
    elif distance < 100:
        return "NEAR"
    elif obj1["x2"] < obj2["x1"]:
        return "TO_THE_LEFT_OF"
    elif obj1["x1"] > obj2["x2"]:
        return "TO_THE_RIGHT_OF"
    return None

# Lưu quan hệ giữa các objects vào CSV
relationships = []
for image_id in df_nodes["image_id"].unique():
    objects = df_nodes[df_nodes["image_id"] == image_id]

    for i in range(len(objects)):
        for j in range(i + 1, len(objects)):
            obj1 = objects.iloc[i]
            obj2 = objects.iloc[j]

            relation = determine_relationship(obj1, obj2)
            if relation:
                relationships.append({
                    "image_id": image_id,
                    "object_1": obj1["node_id"],
                    "object_2": obj2["node_id"],
                    "relationship": relation
                })

df_relationships = pd.DataFrame(relationships)
df_relationships.to_csv("coco_object_relationships.csv", index=False)
print(f"Đã lưu {len(df_relationships)} quan hệ vào: coco_object_relationships.csv")

Đã lưu 220803 quan hệ vào: coco_object_relationships.csv


In [ ]:
# === KẾT NỐI VỚI NEO4J ===

In [7]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

(No data)

In [ ]:
# === XÓA DỮ LIỆU CŨ NEO4J ===

In [12]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [ ]:
# === ĐƯA TOÀN BỘ NODES VÀ RELATIONSHIPS VÀO NEO4J ===

In [11]:
# Load dữ liệu từ CSV
df_objects = pd.read_csv("coco_nodes.csv")
df_relationships = pd.read_csv("coco_object_relationships.csv")

# Dictionary ánh xạ object ID -> Node
node_dict = {}

# Thêm object vào Neo4j với tên thực tế lấy từ label trong CSV
for _, row in df_objects.iterrows():
    object_name = row["label"]  # Sử dụng trực tiếp label làm tên thực tế
    node = Node("Object",
                node_id=int(row["node_id"]),
                name=object_name,  # Dùng label làm tên thực tế
                label=row["label"],
                image_id=int(row["image_id"]),
                x1=row["x1"], y1=row["y1"],
                x2=row["x2"], y2=row["y2"])
    
    graph.merge(node, "Object", "node_id")
    node_dict[row["node_id"]] = node  # Lưu vào dict để tạo quan hệ nhanh hơn

print(f"[✅] Đã thêm {len(df_objects)} objects vào Neo4j với tên thực tế từ label.")

KeyboardInterrupt: 

In [ ]:
# Thêm quan hệ giữa các object
for _, row in df_relationships.iterrows():
    obj1_id = int(row["object_1"])
    obj2_id = int(row["object_2"])
    relationship_type = row["relationship"]

    if obj1_id in node_dict and obj2_id in node_dict:
        rel = Relationship(node_dict[obj1_id], relationship_type, node_dict[obj2_id])
        graph.merge(rel)

print(f"[✅] Đã thêm {len(df_relationships)} quan hệ vào Neo4j.")